In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import VertexAI
from langchain.chains import LLMChain
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession
import re
import time
from transformers import pipeline

c:\Users\Christian\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Initialize Vertex AI
vertexai.init(project='adsp-capstone-once-upon', location = 'us-central1')

# Initialize the LLM
llm = VertexAI(model_name="gemini-1.0-pro", max_output_tokens=8192)

# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

PydanticUserError: `VertexAI` is not fully defined; you should define `_LanguageModel`, then call `VertexAI.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.9/u/class-not-fully-defined

In [5]:
# Define prompt template for the summary
summary_prompt_template = PromptTemplate(
    input_variables=["input_prompt", "user_age"],
    template="""
    Generate a complete summary for a 3-chapter story that is engaging for a {user_age}-year-old child. The story should be about: {input_prompt}

    Your response must include:
    1. A brief introduction to the story
    2. Chapter 1 summary (at least 100 words)
    3. Chapter 2 summary (at least 100 words)
    4. Chapter 3 summary (at least 100 words)

    Format your response as follows:
    Story Introduction: [Your introduction here]

    Chapter 1: [Title]
    [Chapter 1 summary]

    Chapter 2: [Title]
    [Chapter 2 summary]

    Chapter 3: [Title]
    [Chapter 3 summary]

    Ensure each chapter summary is complete and shows a clear progression of the story.
    """
)

# Create LLMChain for summary generation
summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def generate_summary(input_prompt, user_age):
    while True:
        summary = summary_chain.run(input_prompt=input_prompt, user_age=user_age)
        print("Complete Story Summary:")
        print(summary)
        return summary

C:\Users\Christian\AppData\Local\Temp\ipykernel_10552\3265483459.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<vertexai.preview.generat...t at 0x000001E2F58DB150>, input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<vertexai.preview.generat...t at 0x000001E2F58DB150>, input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [8]:
# Define prompt template for chapter generation
#Input needs to include previous chapter summary, theme, character
chapter_prompt_template = PromptTemplate(
    input_variables=["chapter_summary", "chapter_number", "user_age"],
    template="""
    Based on this chapter summary for a {user_age}-year-old child:
    {chapter_summary}

    Generate a detailed Chapter {chapter_number} of AT LEAST 1000 WORDS. The chapter should be written in paragraphs, not bullet points, and should be engaging for a {user_age}-year-old child.

    Do not include any additional notes, just the story that the audience should read. Ensure the chapter matches the summary while expanding on the details and dialog.
    """
)

# Create LLMChain for chapter generation
chapter_chain = LLMChain(llm=llm, prompt=chapter_prompt_template)

def generate_chapter(chapter_summary, chapter_number, user_age):
    chapter = chapter_chain.run(chapter_summary=chapter_summary, chapter_number=chapter_number, user_age=user_age)
    print(f"\nChapter {chapter_number} Generated:")
    print(chapter)
    return chapter

def generate_story_in_chapters(summary_object, user_age):
    chapters = re.split(r'Chapter \d+:', summary_object)[1:]
    generated_chapters = []

    for i, chapter_summary in enumerate(chapters, 1):
        chapter = generate_chapter(chapter_summary, i, user_age)
        generated_chapters.append(chapter)

    return generated_chapters

In [9]:
def chat_loop():
    while True:
        user_age = int(input("Enter the age of the child (e.g., 5): "))
        input_from_user = input("What would you like the story to be about? ")
        
        summary_object = generate_summary(input_from_user, user_age)
        user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        while user_response.upper() == "NO":
            question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
            if question.upper() == "MODIFY":          
                new_prompt = input("Please provide the changes you want to see in this story: ")
                summary_prompt = f"Make the following changes to your story summary, ensuring you maintain complete summaries for all three chapters: {new_prompt}"
                summary_object = summary_chain.run(input_prompt=summary_prompt, user_age=user_age)
                print("\nUpdated Complete Story Summary:")
                print(summary_object)
            elif question.upper() == "NEW STORY":
                input_from_user = input("Let's build you a new story! What would you like to hear a story about? ")
                summary_object = generate_summary(input_from_user, user_age)
            else:
                print("Invalid option. Please choose NEW STORY or MODIFY.")
                continue
            
            user_response = input("Do you want to continue with this summary? (YES/NO): ")

        if user_response.upper() == "YES":
            try:
                generated_chapters = generate_story_in_chapters(summary_object, user_age)
                print("\nFull story generated successfully!")
            except Exception as e:
                print(f"An error occurred: {e}")
                print("Let's try again with a new story idea.")
                continue
        
        continue_response = input("Do you want to create another story? (YES/NO): ")
        if continue_response.upper() != "YES":
            print("Thank you for using the story generator. Goodbye!")
            break

if __name__ == "__main__":
    chat_loop()

Enter the age of the child (e.g., 5):  7
What would you like the story to be about?  A story about Ada Lovelace


Complete Story Summary:
## Story Introduction:

Ada Lovelace was a brilliant young girl who loved numbers and stories. She dreamed of creating a machine that could tell stories using numbers, a machine that could do much more than just add and subtract. Join Ada on her journey as she explores the world of mathematics and invents the first computer program!

## Chapter 1: A Curious Mind

Ada was born into a world of books and ideas. Her mother, a famous writer, encouraged Ada's love for learning. Ada spent hours reading about science, math, and even poetry. But what fascinated her most were the complex machines her father, a renowned inventor, was building. Ada would watch him for hours, her mind buzzing with questions about how they worked. One day, she asked her father, "Can a machine ever think like a human?" Her father smiled and said, "Perhaps, Ada. Perhaps." This sparked a fire in Ada's heart. She was determined to find out.

## Chapter 2: Numbers and Stories

Ada began studying m

Do you want to continue with this summary? (YES/NO):  YES



Chapter 1 Generated:
## Chapter 1: A World of Wonder

Ada Lovelace was born into a world that sparkled with stories and ideas. Her mother, the renowned writer Lady Byron, filled their home with books that whispered tales of faraway lands and brave heroes. Ada, with her bright eyes and insatiable curiosity, devoured these stories, her imagination taking flight with each turn of the page.

But books weren't the only things that captivated Ada. Her father, Lord Byron, was a brilliant inventor, his mind constantly churning with new contraptions and machines. Ada would spend hours in his workshop, watching him tinker and build, her mind buzzing with questions about how these wondrous creations worked.

One day, as Lord Byron was assembling a complex clockwork automaton, Ada, then just five years old, tugged at his sleeve. "Papa," she asked, her voice filled with wonder, "can a machine ever think like a human?"

Lord Byron paused, his eyes twinkling with amusement. "Perhaps, Ada," he said w

KeyboardInterrupt: Interrupted by user

## ***To Check For:***
- If characters extraction function is working
- If chapter 3 has the consolidated summary of chapter 1 & chapter 2

## ***To Add:***
- Ouput Parsing function from LangChain to ensure output format is consistent every time

#  ***Adding Extended input***

In [11]:
# Initialize Vertex AI
vertexai.init(project='adsp-capstone-once-upon', location = 'us-central1')
# Initialize the LLM
llm = VertexAI(model_name="gemini-1.0-pro", max_output_tokens=8192, max_tokens = 32000)
# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

# Define prompt template for the summary
summary_prompt_template = PromptTemplate(
    input_variables=["input_prompt", "user_age"],
    template="""
    Generate a complete summary for a 3-chapter story that is engaging for a {user_age}-year-old child. The story should be about: {input_prompt}
    Your response must include:
    1. A brief introduction to the story
    2. Chapter 1 summary (at least 100 words)
    3. Chapter 2 summary (at least 100 words)
    4. Chapter 3 summary (at least 100 words)
    Format your response as follows:
    Story Introduction: [Your introduction here]
    Chapter 1: [Title]
    [Chapter 1 summary]
    Chapter 2: [Title]
    [Chapter 2 summary]
    Chapter 3: [Title]
    [Chapter 3 summary]
    Ensure each chapter summary is complete and shows a clear progression of the story.
    """
)

# Create LLMChain for summary generation
summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def generate_summary(input_prompt, user_age):
    while True:
        summary = summary_chain.run(input_prompt=input_prompt, user_age=user_age)
        print("Complete Story Summary:")
        print(summary)
        return summary

# Define updated prompt template for chapter generation
chapter_prompt_template = PromptTemplate(
    input_variables=["chapter_title", "previous_summary", "chapter_summary", "characters", "chapter_number", "user_age"],
    template=(
        "This is the outline for the chapter: Title: {chapter_title}.\n"
        "Summary of previous chapters: {previous_summary}.\n"
        "Summary for this chapter: {chapter_summary}.\n"
        "Characters and their associated personalities and behaviors so far: {characters}.\n"
        "Based on this outline, generate a LENGTHY chapter {chapter_number} of AT LEAST 1000 WORD TOKENS. "
        "The story should be written in PARAGRAPHS (not bullet points) and fit for a {user_age}-year-old kid in English. "
        "Please ensure the chapter has AT LEAST 1000 words and does not include any additional notes. "
        "Overall, the total number of tokens should be less than 8000 tokens (all characters included)."
    )
)

# Create LLMChain for chapter generation
chapter_chain = LLMChain(llm=llm, prompt=chapter_prompt_template)

def generate_chapter(chapter_title, previous_summary, chapter_summary, characters, chapter_number, user_age):
    chapter = chapter_chain.run(
        chapter_title=chapter_title,
        previous_summary=previous_summary,
        chapter_summary=chapter_summary,
        characters=characters,
        chapter_number=chapter_number,
        user_age=user_age
    )
    print(f"\nChapter {chapter_number} Generated:")
    print(chapter)
    return chapter

# Add this new prompt template for character extraction
character_extraction_prompt = PromptTemplate(
    input_variables=["story_text"],
    template="""
    Extract the main characters from the following story text, along with a brief description of their personalities and behaviors:

    {story_text}

    Format your response as a list of characters, each with a short description:
    - Character Name: Brief description of personality and behavior
    """
)

# Create LLMChain for character extraction
character_extraction_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Add character extraction function
# Update the extract_characters function
def extract_characters(input_story):
    characters = character_extraction_chain.run(story_text=input_story)
    return characters

def generate_story_in_chapters(summary_object, user_age):
    start_time = time.time()
    
    chapters = re.split(r'Chapter \d+:', summary_object)[1:]
    if len(chapters) != 3:
        raise ValueError("The summary does not contain exactly 3 chapters.")
    
    chapter_structures = [f"Chapter {i+1}:{chapter}" for i, chapter in enumerate(chapters)]
    
    # Generate Chapter 1
    chapter1 = generate_chapter(chapter_structures[0].split(':')[1].strip(), "", chapters[0], "", 1, user_age)
    characters = extract_characters(chapter1)
    chapter1_summary = summarizer(chapter1, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 2
    chapter2 = generate_chapter(chapter_structures[1].split(':')[1].strip(), chapter1_summary, chapters[1], characters, 2, user_age)
    chapter2_summary = summarizer(chapter2, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 3
    previous_summary = f"Chapter 1: {chapter1_summary}\nChapter 2: {chapter2_summary}"
    chapter3 = generate_chapter(chapter_structures[2].split(':')[1].strip(), previous_summary, chapters[2], characters, 3, user_age)
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")
    
    return chapter1, chapter2, chapter3

def chat_loop():
    while True:
        user_age = int(input("Enter the age of the child (e.g., 5): "))
        input_from_user = input("What would you like the story to be about? ")
        
        summary_object = generate_summary(input_from_user, user_age)
        user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        while user_response.upper() == "NO":
            question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
            if question.upper() == "MODIFY":          
                new_prompt = input("Please provide the changes you want to see in this story: ")
                summary_prompt = f"Make the following changes to your story summary, ensuring you maintain complete summaries for all three chapters: {new_prompt}"
                summary_object = summary_chain.run(input_prompt=summary_prompt, user_age=user_age)
                print("\nUpdated Complete Story Summary:")
                print(summary_object)
            elif question.upper() == "NEW STORY":
                input_from_user = input("Let's build you a new story! What would you like to hear a story about? ")
                summary_object = generate_summary(input_from_user, user_age)
            else:
                print("Invalid option. Please choose NEW STORY or MODIFY.")
                continue
            
            user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        if user_response.upper() == "YES":
            try:
                generated_chapters = generate_story_in_chapters(summary_object, user_age)
                print("\nFull story generated successfully!")
            except Exception as e:
                print(f"An error occurred: {e}")
                print("Let's try again with a new story idea.")
                continue
        
        continue_response = input("Do you want to create another story? (YES/NO): ")
        if continue_response.upper() != "YES":
            print("Thank you for using the story generator. Goodbye!")
            break

if __name__ == "__main__":
    chat_loop()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Enter the age of the child (e.g., 5):  6
What would you like the story to be about?  A story about 4 friends and a dog


Complete Story Summary:
## Story Introduction:

Four best friends, Maya, Leo, Ben, and Chloe, lived on a quiet street filled with laughter and sunshine. They were inseparable, always embarking on exciting adventures together. One sunny afternoon, while playing in Maya's backyard, they stumbled upon a curious sight – a small, fluffy dog hiding under the bushes.

## Chapter 1: A Mysterious New Friend

The friends cautiously approached the dog, who whimpered and trembled. Maya, known for her gentle nature, knelt down and offered a piece of her sandwich. The dog, with hesitant steps, approached and licked her hand. They named him Lucky, for he seemed to have found his lucky day.

## Chapter 2: The Great Treasure Hunt

Lucky, full of energy and playful spirit, quickly became part of the gang. One day, while exploring the nearby woods, they discovered an old, dusty map. It depicted a hidden treasure buried somewhere within the woods. Armed with shovels and boundless enthusiasm, the friends, 

Do you want to continue with this summary? (YES/NO):  yes



Chapter 1 Generated:
## Chapter 1: A Mysterious New Friend

The sun peeked over the horizon, painting the sky with vibrant hues of orange and pink. The air was crisp and cool, carrying the sweet scent of wildflowers. Maya, Leo, and Ben, three inseparable friends, skipped along the dusty path, their backpacks bouncing merrily on their shoulders. They were on their way to their favorite spot by the river, a hidden haven where they could spend hours building sandcastles, chasing butterflies, and sharing stories.

As they approached the riverbank, a faint whimpering caught their attention. They stopped, their eyes scanning the area. There, huddled beneath a willow tree, was a small, brown dog. His fur was matted and dirty, and his tail tucked between his legs. He looked scared and alone.

"Oh, poor thing," Maya whispered, her heart melting with compassion. "He must be lost."

Leo, the adventurous one, took a step forward. "Let's go see him," he said, his voice filled with curiosity.

Ben,

Do you want to create another story? (YES/NO):  no


Thank you for using the story generator. Goodbye!


In [4]:
print(llm)

VertexAI
Params: {'model_name': 'gemini-1.0-pro', 'temperature': 0.0, 'max_output_tokens': 8192, 'candidate_count': 1, 'top_k': 40, 'top_p': 0.95}


In [10]:
import langchain
print(langchain.__version__)

0.1.16


### ***

## ***Adding Output Parser for Structure and Consistency***

In [1]:
import re
import time
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
#from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
from langchain.chains import LLMChain
from transformers import pipeline
import vertexai

# Initialize Vertex AI
vertexai.init(project='adsp-capstone-once-upon', location='us-central1')

# Initialize the LLM
llm = VertexAI(model_name="gemini-1.0-pro", max_output_tokens=8192, max_tokens=32000)

# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

# Define response schemas for structured output
summary_response_schemas = [
    ResponseSchema(name="introduction", description="A brief introduction to the story"),
    ResponseSchema(name="chapter_1", description="Summary of Chapter 1 (at least 100 words)"),
    ResponseSchema(name="chapter_2", description="Summary of Chapter 2 (at least 100 words)"),
    ResponseSchema(name="chapter_3", description="Summary of Chapter 3 (at least 100 words)"),
]

summary_output_parser = StructuredOutputParser.from_response_schemas(summary_response_schemas)

# Update the summary prompt template
summary_prompt_template = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """Generate a complete summary for a 3-chapter story that is engaging for a {user_age}-year-old child. The story should be about: {input_prompt}

{format_instructions}

Ensure each chapter summary is complete and shows a clear progression of the story."""
        )
    ],
    input_variables=["input_prompt", "user_age"],
    partial_variables={"format_instructions": summary_output_parser.get_format_instructions()}
)

# Create LLMChain for summary generation
summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def generate_summary(input_prompt, user_age):
    summary = summary_chain.run(input_prompt=input_prompt, user_age=user_age)
    parsed_summary = summary_output_parser.parse(summary)
    print("Complete Story Summary:")
    print(f"Introduction: {parsed_summary['introduction']}")
    print(f"Chapter 1: {parsed_summary['chapter_1']}")
    print(f"Chapter 2: {parsed_summary['chapter_2']}")
    print(f"Chapter 3: {parsed_summary['chapter_3']}")
    return parsed_summary

# Define response schemas for chapter generation
chapter_response_schemas = [
    ResponseSchema(name="content", description="The full content of the chapter"),
    ResponseSchema(name="word_count", description="The word count of the chapter"),
]

chapter_output_parser = StructuredOutputParser.from_response_schemas(chapter_response_schemas)

# Update the chapter prompt template
chapter_prompt_template = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """This is the outline for the chapter: Title: {chapter_title}.
Summary of previous chapters: {previous_summary}.
Summary for this chapter: {chapter_summary}.
Characters and their associated personalities and behaviors so far: {characters}.

Based on this outline, generate a LENGTHY chapter {chapter_number} of AT LEAST 1000 WORD TOKENS.
The story should be written in PARAGRAPHS (not bullet points) and fit for a {user_age}-year-old kid in English.
Please ensure the chapter has AT LEAST 1000 words and does not include any additional notes.
Overall, the total number of tokens should be less than 8000 tokens (all characters included).

{format_instructions}"""
        )
    ],
    input_variables=["chapter_title", "previous_summary", "chapter_summary", "characters", "chapter_number", "user_age"],
    partial_variables={"format_instructions": chapter_output_parser.get_format_instructions()}
)

# Create LLMChain for chapter generation
chapter_chain = LLMChain(llm=llm, prompt=chapter_prompt_template)

def generate_chapter(chapter_title, previous_summary, chapter_summary, characters, chapter_number, user_age):
    chapter = chapter_chain.run(
        chapter_title=chapter_title,
        previous_summary=previous_summary,
        chapter_summary=chapter_summary,
        characters=characters,
        chapter_number=chapter_number,
        user_age=user_age
    )
    parsed_chapter = chapter_output_parser.parse(chapter)
    print(f"\nChapter {chapter_number} Generated:")
    print(f"Word count: {parsed_chapter['word_count']}")
    print(parsed_chapter['content'])
    return parsed_chapter

# Update the character extraction prompt and parser
character_response_schemas = [
    ResponseSchema(name="characters", description="List of main characters with their descriptions")
]

character_output_parser = StructuredOutputParser.from_response_schemas(character_response_schemas)

character_extraction_prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """Extract the main characters from the following story text, along with a brief description of their personalities and behaviors:

{story_text}

{format_instructions}

Format your response as a list of characters, each with a short description:
- Character Name: Brief description of personality and behavior"""
        )
    ],
    input_variables=["story_text"],
    partial_variables={"format_instructions": character_output_parser.get_format_instructions()}
)

# Create LLMChain for character extraction
character_extraction_chain = LLMChain(llm=llm, prompt=character_extraction_prompt)

# Update the extract_characters function
def extract_characters(input_story):
    characters = character_extraction_chain.run(story_text=input_story)
    parsed_characters = character_output_parser.parse(characters)
    return parsed_characters['characters']

def generate_story_in_chapters(summary_object, user_age):
    start_time = time.time()
    
    # Generate Chapter 1
    chapter1 = generate_chapter("Chapter 1", "", summary_object['chapter_1'], "", 1, user_age)
    characters = extract_characters(chapter1['content'])
    chapter1_summary = summarizer(chapter1['content'], max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 2
    chapter2 = generate_chapter("Chapter 2", chapter1_summary, summary_object['chapter_2'], characters, 2, user_age)
    chapter2_summary = summarizer(chapter2['content'], max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    
    # Generate Chapter 3
    previous_summary = f"Chapter 1: {chapter1_summary}\nChapter 2: {chapter2_summary}"
    chapter3 = generate_chapter("Chapter 3", previous_summary, summary_object['chapter_3'], characters, 3, user_age)
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")
    
    return chapter1, chapter2, chapter3

def chat_loop():
    while True:
        user_age = int(input("Enter the age of the child (e.g., 5): "))
        input_from_user = input("What would you like the story to be about? ")
        
        summary_object = generate_summary(input_from_user, user_age)
        user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        while user_response.upper() == "NO":
            question = input("Do you want a new story or make changes to this storyline? (NEW STORY/MODIFY): ")
            if question.upper() == "MODIFY":          
                new_prompt = input("Please provide the changes you want to see in this story: ")
                summary_prompt = f"Make the following changes to your story summary, ensuring you maintain complete summaries for all three chapters: {new_prompt}"
                summary_object = generate_summary(summary_prompt, user_age)
            elif question.upper() == "NEW STORY":
                input_from_user = input("Let's build you a new story! What would you like to hear a story about? ")
                summary_object = generate_summary(input_from_user, user_age)
            else:
                print("Invalid option. Please choose NEW STORY or MODIFY.")
                continue
            
            user_response = input("Do you want to continue with this summary? (YES/NO): ")
        
        if user_response.upper() == "YES":
            try:
                generated_chapters = generate_story_in_chapters(summary_object, user_age)
                print("\nFull story generated successfully!")
                # Here you can add code to save or further process the generated chapters
            except Exception as e:
                print(f"An error occurred: {e}")
                print("Let's try again with a new story idea.")
                continue
        
        continue_response = input("Do you want to create another story? (YES/NO): ")
        if continue_response.upper() != "YES":
            print("Thank you for using the story generator. Goodbye!")
            break

if __name__ == "__main__":
    chat_loop()

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/opt/conda/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)

In [2]:
import tensorflow as tf
import google.protobuf

print("TensorFlow version:", tf.__version__)
print("protobuf version:", google.protobuf.__version__)


2024-10-15 20:02:50.779010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 20:02:52.327685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-10-15 20:02:52.327878: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

TensorFlow version: 2.11.0
protobuf version: 3.19.0


In [8]:
import tensorflow

print(tensorflow.__version__)

2.18.0-rc1


In [9]:
!pip uninstall tensorflow transformers -y

Found existing installation: tensorflow 2.18.0rc1
Uninstalling tensorflow-2.18.0rc1:
  Successfully uninstalled tensorflow-2.18.0rc1
Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0


In [10]:
pip install tensorflow==2.11.0 transformers==4.41.0

  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (787 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.7 MB/s eta 0:00:00:00:0100:01
Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:000:00:01
Using cached protobuf-3.19.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 61.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstal

In [3]:
!pip uninstall protobuf -y

!pip install protobuf==3.19.0


Found existing installation: protobuf 3.19.6
Uninstalling protobuf-3.19.6:
  Successfully uninstalled protobuf-3.19.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.26.1
    Uninstalling protobuf-5.26.1:
      Successfully uninstalled protobuf-5.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have p

In [14]:
pip list | grep -E "pydantic|langchain|langchain-core"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


langchain                                0.3.3
langchain-community                      0.0.34
langchain-core                           0.3.10
langchain-google-vertexai                1.0.3
langchain-text-splitters                 0.3.0
pydantic                                 2.9.2
pydantic_core                            2.23.4
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install --upgrade langchain langchain-core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [13]:
import tensorflow as tf
print(tensorflow.__version__)

ImportError: /opt/conda/lib/python3.10/site-packages/tensorflow/python/../libtensorflow_cc.so.2: undefined symbol: _ZN4toco9TocoFlags5Impl_66_i_give_permission_to_break_this_code_default_qdq_conversion_mode_E

In [7]:
history

import re
import time
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
#from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
from langchain.chains import LLMChain
from transformers import pipeline
import vertexai

# Initialize Vertex AI
vertexai.init(project='adsp-capstone-once-upon', location='us-central1')

# Initialize the LLM
llm = VertexAI(model_name="gemini-1.0-pro", max_output_tokens=8192, max_tokens=32000)

# Initialize the text summarization pipeline
summarizer = pipeline("summarization")

# Define response schemas for structured output
summary_response_schemas = [
    ResponseSchema(name="introduction", description="A brief introduction to the story"),
    ResponseSchema(name="chapter_1", description="Summary of Chapter 1 (at least 100 words)"),
    ResponseSchema(name="chapter_2", description="Summary of Chapter 2 (at least 100 words)"),
    R

In [3]:
!pip uninstall keras -y
!pip install tensorflow 

Found existing installation: keras 3.6.0
Uninstalling keras-3.6.0:
  Successfully uninstalled keras-3.6.0

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [4]:
!pip install tensorflow 

  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.6.0-py3-none-any.whl (1.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.5 which is incompatible.


In [1]:
import tensorflow as tf
from tensorflow import keras

2024-10-15 19:47:18.662120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1729021638.690922   34581 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1729021638.699150   34581 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 19:47:18.728656: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
